# Example joint control with BLF in Python

## Introduction

### Background

The code in this example is written in Python and uses the [Bipedal Locomotion Framework](https://github.com/ami-iit/bipedal-locomotion-framework) (BLF) to control the joints of the [ergoCub](https://ergocub.eu/project) robot in the Gazebo simulator. To run the code in this example you need to have a working installation of the YARP middleware and the Gazebo simulator, as well as the BLF and ergoCub software. An easy way to install all the dependencies is to use the conda environment that can be created with the following command:

```bash
conda env create -n blf_example_env gazebo-yarp-plugins icub-models ergocub-software bopedal-locomotion-framework jupyter
```

this environment should contain all the dependencies needed to run the example.

In all terminal windows, activate the conda environment:
```bash
conda activate blf_example_env
```


In [ ]:
import numpy as np
import yarp
import tempfile

import bipedal_locomotion_framework.bindings as blf

### Yarp server

Terminal 1:
```bash
export YARP_ROBOT_NAME="ergoCubGazeboV1_1"
yarpserver --write
``` 

This will start the YARP name server that allows the different processes to communicate with each other.

At this point we can connect to the yarp network.

In [ ]:
network = yarp.Network()

### Gazebo

Terminal 2:
```bash
export YARP_ROBOT_NAME="ergoCubGazeboV1_1"
export YARP_CLOCK=/clock
gazebo -slibgazebo_yarp_clock.so
```

After Gazebo is open, drag and drop the robot model 'ergoCubGazeboV1_1' from the left panel Insert tab to the Gazebo world.

# Configuration files

A minimal configuration file is shown in the next cell. It has two groups, one for the robot control boards and one for the sensor bridge. The control board group will be used to intialize the access to the joint motors, while the sensor bridge group will be used to access the joint encoders.

In [ ]:
config = """
[REMOTE_CONTROL_BOARD]
robot_name ergocubSim

remote_control_boards (left_arm)

joints_list (l_shoulder_pitch l_shoulder_roll l_shoulder_yaw l_elbow l_wrist_yaw l_wrist_pitch l_wrist_roll)

positioning_duration 10.0 # seconds
positioning_tolerance 0.1 # radians
position_direct_max_admissible_error 0.1 # radians

[SENSOR_BRIDGE]
check_for_nan                                    false
stream_joint_states                              true
stream_motor_states                              false
stream_forcetorque_sensors                       false
"""

tmpfile = tempfile.NamedTemporaryFile(mode='w', delete=False)
tmpfile.write(config)
tmpfile.close()

We have now a configuration file in a temporary file, let's use it with the `YarpParametersHandler` class to read the configuration file and print the parameters.

In [ ]:
param_handler = blf.parameters_handler.YarpParametersHandler()
param_handler.set_from_filename(tmpfile.name)

# add the local prefix to the remote control board group
# this is needed for the robot control
param_handler.get_group("REMOTE_CONTROL_BOARD").set_parameter_string("local_prefix", "example_controller")

print(param_handler)

## Motor Control

To gain access to the robot control board we need to instantiate a PolyDriver object and pass it the configuration file. This will allow us to access the control board remote control interface.

Refs:
 - https://www.yarp.it/latest/classRemoteControlBoardRemapper.html
 - https://ami-iit.github.io/bipedal-locomotion-framework/YarpHelper_8h.html#a2c07f5099140671abde506a8a316e130
 - https://ami-iit.github.io/bipedal-locomotion-framework/structBipedalLocomotion_1_1RobotInterface_1_1PolyDriverDescriptor.html

In [ ]:
polydrivers = {}
polydrivers["REMOTE_CONTROL_BOARD"] = blf.robot_interface.construct_remote_control_board_remapper(param_handler.get_group("REMOTE_CONTROL_BOARD"))
polydrivers["REMOTE_CONTROL_BOARD"].is_valid()

After instantiating the PolyDriver object we can create a YarpRobotContol object to access the joints specified in the configuration file.

In [ ]:
robot_control = blf.robot_interface.YarpRobotControl()
robot_control.initialize(param_handler.get_group("REMOTE_CONTROL_BOARD"))
robot_control.set_driver(polydrivers["REMOTE_CONTROL_BOARD"].poly)

At this point we have access to the robot joints, for example to count the number of dofs.

In [ ]:
n_dof = len(robot_control.get_joint_list())
n_dof

We can set the control mode of the joints to position direct and set the desired position of the joints.

In [ ]:
robot_control.set_control_mode(blf.robot_interface.YarpRobotControl.PositionDirect)

In [ ]:
joint_values = np.zeros(n_dof)
control_modes = blf.robot_interface.YarpRobotControl.PositionDirect

robot_control.set_references(joint_values, control_modes)

If the requested joint position is greater than the parameter `position_direct_max_admissible_error` (in rad) then the `set_reference` function will return false and log an error message.
To avoid this we can set the parameter `position_direct_max_admissible_error` to a higher value, or pass the third argument of the `set_reference` function which is the current joint position read from the encoders.

We can use `check_motion_done` to check if the motion is done. The function returns a tuple of 4 values:
1) isOk: True if the function executed correctly, False otherwise.
2) motionDone: True if the motion ended, False otherwise.
3) isTimeExpired: True if internal timer expired, False otherwise.
4) info: vector containing the list of the joint whose motion did not finish yet, and the corresponding position error.


In [ ]:
robot_control.check_motion_done()

# Sensor Readings



To control the robot in closed loop (hopefully that is the goal) we need to read the joint encoders. We can use the `YarpSensorBridge` class to access the joint encoders.

Let's create a `YarpSensorBridge` object and initialize it with the relative group in the configuration file.

In [ ]:
sensor_bridge = blf.robot_interface.YarpSensorBridge()

sensor_bridge.initialize(param_handler.get_group("SENSOR_BRIDGE"))

To have access to the joint encoders we need to link the `PolyDriver` object to the `YarpSensorBridge` object.
This will allow us to read encoders of the joints we are controlling and that we defined in the `REMOTE_CONTROL_BOARD` group of the configuration file.

In [ ]:
sensor_bridge.set_drivers_list([polydrivers["REMOTE_CONTROL_BOARD"]])

At this point we can read the joint encoders and print them.

In [ ]:
sensor_bridge.advance()
are_joints_ok, joint_positions, _ = sensor_bridge.get_joint_positions()
joint_positions

# Example Control

We now have all the tools to control the robot. We can create a simple control loop that moves the joints to a desired position.

In [ ]:
joint_to_test = "l_shoulder_pitch"
joint_index = robot_control.get_joint_list().index(joint_to_test)

In [ ]:
for i in range(100):
    t = i * 0.1
    # read the joint positions
    sensor_bridge.advance()
    are_joints_ok, joint_positions, _ = sensor_bridge.get_joint_positions()

    new_joint_positions = joint_positions.copy()
    # set the references
    new_joint_positions[joint_index] += 0.1 * np.cos(10 * t)
    robot_control.set_references(new_joint_positions, blf.robot_interface.YarpRobotControl.PositionDirect, joint_positions)
    yarp.delay(0.1)